In [1]:
import pandas as pd
from keras.layers import Conv1D, GRU, Dense, Dropout, Flatten
from keras.models import Sequential
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import train_test_split, GridSearchCV

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
cleared_data_all = pd.read_excel(r'dataset/cleared_dataset/BRICS_all_areas_cleared_dataset.xlsx')
cleared_data_all = cleared_data_all.iloc[:, 1:]
cleared_data_all

,Adjusted net national income (annual % growth),Adjusted net national income (constant 2015 US$),Adjusted net national income (current US$),Adjusted net national income per capita (annual % growth),Adjusted net national income per capita (constant 2015 US$),Adjusted net national income per capita (current US$),"Adjusted net savings, excluding particulate emission damage (current US$)","Adjusted net savings, including particulate emission damage (current US$)",Adjusted savings: carbon dioxide damage (current US$),Adjusted savings: consumption of fixed capital (current US$),...,"Unemployment, youth male (% of male labor force ages 15-24) (modeled ILO estimate)","Unemployment, youth male (% of male labor force ages 15-24) (national estimate)","Unemployment, youth total (% of total labor force ages 15-24) (modeled ILO estimate)","Unemployment, youth total (% of total labor force ages 15-24) (national estimate)","Vulnerable employment, female (% of female employment) (modeled ILO estimate)","Vulnerable employment, male (% of male employment) (modeled ILO estimate)","Vulnerable employment, total (% of total employment) (modeled ILO estimate)","Wage and salaried workers, female (% of female employment) (modeled ILO estimate)","Wage and salaried workers, male (% of male employment) (modeled ILO estimate)","Wage and salaried workers, total (% of total employment) (modeled ILO estimate)"
0,0.484076,0.018315,0.001125,0.493361,0.232096,0.017700,0.025495,0.037758,0.000056,0.000635,...,0.108886,0.023856,0.076645,0.056228,0.279695,0.201838,0.156617,0.323412,0.336784,0.355807
1,0.468048,0.014754,0.001301,0.482257,0.223844,0.014627,0.026590,0.037826,0.000128,0.000285,...,0.080654,0.003790,0.109930,0.043929,0.193565,0.248208,0.176652,0.378192,0.258009,0.438305
2,0.481640,0.017279,0.000530,0.473711,0.250464,0.014431,0.026371,0.038822,0.000118,0.000204,...,0.091330,0.033513,0.087172,0.036492,0.235089,0.298487,0.198736,0.409490,0.283037,0.495832
3,0.453289,0.016555,0.001096,0.499597,0.228257,0.023174,0.026353,0.034463,0.000179,0.000457,...,0.052015,0.022731,0.091999,0.035654,0.245976,0.240291,0.310167,0.292724,0.318325,0.516666
4,0.470644,0.022188,0.000600,0.508247,0.202923,0.022862,0.026775,0.030509,0.000266,0.000555,...,0.044705,0.007773,0.118308,0.038628,0.240441,0.223383,0.363340,0.317195,0.347051,0.452032
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2027,0.355408,0.021423,0.021071,0.376423,0.394665,0.319516,0.022198,0.021916,0.046591,0.021313,...,0.786150,0.826007,0.640357,0.728138,0.083754,0.051664,0.078857,0.482987,0.446748,0.472202
2028,0.355226,0.015958,0.012176,0.379859,0.443141,0.318189,0.022149,0.022046,0.043437,0.015338,...,0.525959,0.723408,0.744625,0.823676,0.091201,0.047630,0.097655,0.475970,0.344713,0.593125
2029,0.355856,0.020443,0.021131,0.377490,0.433445,0.300040,0.022193,0.021948,0.045275,0.021177,...,0.470119,0.819686,0.710154,0.540721,0.085964,0.065204,0.085008,0.599174,0.528091,0.484817
2030,0.356276,0.017869,0.021781,0.377799,0.392040,0.331834,0.022217,0.021949,0.036685,0.021008,...,0.553846,0.799002,0.719372,0.580822,0.081256,0.063866,0.049129,0.505235,0.456719,0.395095


In [4]:
cleared_data_only_percent = pd.read_excel(r'dataset/cleared_dataset/BRICS_all_areas_cleared_dataset_only_%.xlsx')
cleared_data_only_percent = cleared_data_only_percent.iloc[:, 1:]
cleared_data_only_percent

,Adjusted net national income (annual % growth),Adjusted net national income per capita (annual % growth),"Agriculture, forestry, and fishing, value added (annual % growth)",Chemicals (% of value added in manufacturing),"Communications, computer, etc. (% of service exports, BoP)","Communications, computer, etc. (% of service imports, BoP)","Debt service (PPG and IMF only, % of exports of goods, services and primary income)",Exports of goods and services (annual % growth),Final consumption expenditure (annual % growth),"Food, beverages and tobacco (% of value added in manufacturing)",...,"Unemployment, youth male (% of male labor force ages 15-24) (modeled ILO estimate)","Unemployment, youth male (% of male labor force ages 15-24) (national estimate)","Unemployment, youth total (% of total labor force ages 15-24) (modeled ILO estimate)","Unemployment, youth total (% of total labor force ages 15-24) (national estimate)","Vulnerable employment, female (% of female employment) (modeled ILO estimate)","Vulnerable employment, male (% of male employment) (modeled ILO estimate)","Vulnerable employment, total (% of total employment) (modeled ILO estimate)","Wage and salaried workers, female (% of female employment) (modeled ILO estimate)","Wage and salaried workers, male (% of male employment) (modeled ILO estimate)","Wage and salaried workers, total (% of total employment) (modeled ILO estimate)"
0,0.457406,0.453062,0.569723,0.657691,0.204321,0.303860,0.342552,0.572192,0.932416,0.255100,...,0.068611,0.047416,0.111070,0.056180,0.229610,0.294200,0.233323,0.343718,0.338674,0.365386
1,0.457128,0.470747,0.605518,0.697599,0.379046,0.277715,0.326866,0.571897,0.970064,0.291585,...,0.067162,0.054393,0.083389,0.069938,0.260012,0.235632,0.294720,0.268808,0.404593,0.345781
2,0.513473,0.452032,0.639689,0.574743,0.386501,0.228170,0.400955,0.559396,0.916655,0.282468,...,0.084913,0.062121,0.088835,0.078727,0.228531,0.239684,0.268762,0.360262,0.354494,0.379005
3,0.524539,0.472299,0.610210,0.573498,0.448555,0.245257,0.407111,0.549325,0.827048,0.278518,...,0.105380,0.039375,0.094489,0.041022,0.202504,0.259351,0.266026,0.298337,0.474548,0.358246
4,0.539012,0.476936,0.619483,0.490571,0.540174,0.403026,0.375958,0.539631,0.742310,0.354712,...,0.105057,0.038862,0.093814,0.024142,0.306630,0.265141,0.229772,0.354893,0.295688,0.201926
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2027,0.380864,0.344909,0.564977,0.023685,0.367445,0.537930,0.142920,0.388038,0.306280,0.802443,...,0.575320,0.676519,0.781112,0.671105,0.093923,0.071851,0.056182,0.350254,0.506756,0.504535
2028,0.379569,0.343628,0.611090,0.088613,0.421711,0.715505,0.217099,0.386017,0.277552,0.943814,...,0.423149,0.647875,0.941687,0.748946,0.091576,0.075240,0.075146,0.441824,0.619265,0.562003
2029,0.379445,0.345375,0.617299,0.077532,0.320089,0.629314,0.220856,0.357104,0.256545,0.750134,...,0.654935,0.719199,0.776202,0.660155,0.096665,0.074015,0.083483,0.422319,0.487406,0.472707
2030,0.379651,0.341619,0.688184,0.069574,0.398203,0.801984,0.253660,0.367616,0.194880,0.752336,...,0.528392,0.576105,0.519103,0.780148,0.092425,0.074955,0.063195,0.507861,0.401599,0.612767


Division into training and test samples

In [5]:
test_size = 0.2
seed = 7
target_feature_name = 'GDP per capita (current US$)'

data_X_all = cleared_data_all.drop([target_feature_name], axis=1)
data_Y_all = cleared_data_all[target_feature_name]
data_X_train_all, data_X_test_all, data_Y_train_all, data_Y_test_all = train_test_split(data_X_all, data_Y_all, test_size=test_size, random_state=seed)

data_X_only_percent = cleared_data_only_percent.drop([target_feature_name], axis=1)
data_Y_only_percent = cleared_data_only_percent[target_feature_name]
data_X_train_only_percent, data_X_test_only_percent, data_Y_train_only_percent, data_Y_test_only_percent = train_test_split(data_X_only_percent, data_Y_only_percent, test_size=test_size, random_state=seed)

datasets = [{'X_train': data_X_train_only_percent, 'X_test': data_X_test_only_percent, 'Y_train': data_Y_train_only_percent,
             'Y_test': data_Y_test_only_percent, 'name': 'only_percent_dataset'},
            {'X_train': data_X_train_all, 'X_test': data_X_test_all,
             'Y_train': data_Y_train_all, 'Y_test':data_Y_test_all, 'name': 'original_data'}]

In [6]:
datasets[0].get('X_test')

,Adjusted net national income (annual % growth),Adjusted net national income per capita (annual % growth),"Agriculture, forestry, and fishing, value added (annual % growth)",Chemicals (% of value added in manufacturing),"Communications, computer, etc. (% of service exports, BoP)","Communications, computer, etc. (% of service imports, BoP)","Debt service (PPG and IMF only, % of exports of goods, services and primary income)",Exports of goods and services (annual % growth),Final consumption expenditure (annual % growth),"Food, beverages and tobacco (% of value added in manufacturing)",...,"Unemployment, youth male (% of male labor force ages 15-24) (modeled ILO estimate)","Unemployment, youth male (% of male labor force ages 15-24) (national estimate)","Unemployment, youth total (% of total labor force ages 15-24) (modeled ILO estimate)","Unemployment, youth total (% of total labor force ages 15-24) (national estimate)","Vulnerable employment, female (% of female employment) (modeled ILO estimate)","Vulnerable employment, male (% of male employment) (modeled ILO estimate)","Vulnerable employment, total (% of total employment) (modeled ILO estimate)","Wage and salaried workers, female (% of female employment) (modeled ILO estimate)","Wage and salaried workers, male (% of male employment) (modeled ILO estimate)","Wage and salaried workers, total (% of total employment) (modeled ILO estimate)"
1997,0.379939,0.345840,0.414133,0.060891,0.107854,0.459784,0.090567,0.504744,0.384760,0.507845,...,0.665714,0.599551,0.488661,0.587064,0.079933,0.089474,0.056369,0.545199,0.627403,0.783741
1473,0.409865,0.389454,0.666971,0.202139,0.343054,0.508003,0.241341,0.557378,0.576394,0.337408,...,0.157239,0.258168,0.178792,0.240563,0.065208,0.062799,0.063338,0.546864,0.596952,0.778758
1780,0.412347,0.366612,0.623417,0.347286,0.105018,0.215113,0.130286,0.558712,0.460795,0.259304,...,0.366484,0.525873,0.450933,0.441714,0.099356,0.054689,0.058295,0.595453,0.694256,0.457612
1549,0.497800,0.457875,0.632391,0.195915,0.433418,0.270878,0.086157,0.521833,0.517588,0.239595,...,0.168859,0.143551,0.160175,0.174767,0.041150,0.048664,0.050554,0.382182,0.412283,0.503545
386,0.372307,0.343724,0.498267,0.444890,0.595595,0.629335,0.257082,0.506804,0.427091,0.575212,...,0.183975,0.303696,0.280879,0.311169,0.248090,0.234288,0.245396,0.413345,0.422716,0.581241
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,0.404500,0.369823,0.494723,0.306444,0.460227,0.301380,0.364318,0.639038,0.443514,0.570523,...,0.108111,0.130971,0.134081,0.163679,0.219465,0.290168,0.224745,0.392577,0.384062,0.404236
98,0.377329,0.337415,0.474569,0.687258,0.365539,0.377954,0.560451,0.494554,0.472849,0.544441,...,0.087941,0.062531,0.070826,0.046332,0.199569,0.309762,0.199383,0.330373,0.389821,0.259679
768,0.451207,0.467682,0.540354,0.281106,0.589321,0.185947,0.018965,0.855197,0.652724,0.338606,...,0.077229,0.141973,0.071229,0.141566,0.289573,0.428931,0.314816,0.244636,0.136123,0.376732
610,0.529349,0.479712,0.563440,0.232422,0.210641,0.284510,0.133657,0.697003,0.737674,0.279563,...,0.036062,0.160161,0.016273,0.305178,0.479119,0.446903,0.603112,0.209804,0.144361,0.181629


Serial connection one-dimensional convolutional and recurrent GRU neural network layers with a fully connected classifier

In [7]:
param_grid_neural_network = {
    'filters': [8, 32],
    'kernel_size': [1, 3],
    'units': [16, 64],
    'optimizer' : ["adam", "rmsprop"],
    'dropout_rate': [0.1, 0.2, 0.3],
    'gru_act': ['tanh', 'relu'],
    'conv_act': ['tanh', 'relu']
}

In [8]:
def create_model(filters, kernel_size, units, dropout_rate, optimizer_model, gru_act, conv_act):
    input_shape = (dataset.get('X_train').shape[1], 1)
    model = Sequential()
    model.add(Conv1D(filters=filters, kernel_size=kernel_size, activation=conv_act, input_shape=input_shape))
    model.add(GRU(units, input_shape=input_shape, activation=gru_act))
    model.add(Dropout(rate=dropout_rate))
    model.add(Flatten())
    model.add(Dense(1))
    model.compile(optimizer=optimizer_model, loss='mean_squared_error')
    return model

best_params = []
mse = []

for dataset in datasets:
    regressor = KerasRegressor(build_fn=create_model, filters=32, kernel_size=3, units=64, dropout_rate=0.2, optimizer_model='adam', gru_act='tanh', conv_act='relu', epochs=3, batch_size=32)
    grid = GridSearchCV(estimator=regressor, param_grid=param_grid_neural_network, n_jobs=7)
    grid_result = grid.fit(dataset.get('X_train'), dataset.get('Y_train'), validation_data = (dataset.get('X_test'), dataset.get('Y_test')))

    best_params.append(grid_result.best_params_)
    print("Лучшее значение параметров: ", grid_result.best_params_)

    score = grid.score(dataset.get('X_test'), dataset.get('Y_test'))
    mse.append(score)
    print("MSE:", score)

print(best_params)
print(mse)



Epoch 1/3

51/51 [==============================] - 16s 278ms/step - loss: 0.0254 - val_loss: 0.0196
Epoch 2/3
51/51 [==============================] - 14s 268ms/step - loss: 0.0204 - val_loss: 0.0192
Epoch 3/3
51/51 [==============================] - 14s 268ms/step - loss: 0.0202 - val_loss: 0.0193
Лучшее значение параметров:  {'conv_act': 'relu', 'dropout_rate': 0.2, 'filters': 32, 'gru_act': 'tanh', 'kernel_size': 3, 'optimizer': 'adam', 'units': 64}
13/13 [==============================] - 1s 68ms/step
MSE: 0.2362907928344532
Epoch 1/3
51/51 [==============================] - 40s 755ms/step - loss: 0.0351 - val_loss: 0.0249
Epoch 2/3
51/51 [==============================] - 32s 626ms/step - loss: 0.0244 - val_loss: 0.0237
Epoch 3/3
51/51 [==============================] - 26s 518ms/step - loss: 0.0238 - val_loss: 0.0229
Лучшее значение параметров:  {'conv_act': 'relu', 'dropout_rate': 0.1, 'filters': 32, 'gru_act': 'relu', 'kernel_size': 3, 'optimizer': 'adam', 'units': 64}
13/13